### Imports

In [1]:
# Source
# https://www.quantopian.com/research/notebooks/S13%20L109%20-%20Advanced%20Quantopian%20%26%20Trading%20Algorithms.ipynb
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import Q1500US

### Build an Empty Pipeline with a Screen
In the code below create an empty pipeline with a screen that filters the pipeline output for stocks with a 60-day average dollar volume greater then $50,000

In [2]:
# Source - Udacity Factors L21: 13
from quantopian.pipeline.factors import AverageDollarVolume

# Create a screen for our pipeline
universe = AverageDollarVolume(window_length = 60) > 50000
universe

# create an empty pipeline with the given screen
pipeline = Pipeline(screen = universe)
pipeline

### Create a Pipeline Engine
In the code below, create a data loader that loads oru data bundle using Zipline's `USEquityPricingLoader`. Then create a function to be passed to the `get_loader` parameter in the pipline engine. Using the NYSE trading calendar create a pipeline engine using Zipline's `SimplePipelineEngine`.

In [3]:
# import resources
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
# from zipline.pipline.engine import SimplePipelineEngine
from pipline.engine import SimplePipelineEngine
from zipline.pipline.loaders import USEquityPricingLoader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

SandboxViolation: Importing SimplePipelineEngine from pipline.engine raised an ImportError. No modules or attributes with a similar name were found.

## Create Factors and Filters for the pipeline
### Factors
In the code below, create 2 factors:
1. **Custom Factor:** Create a custom factor, `percent_difference`, by combining a 30-day average closing price factor and a 60-day average closing price factor. The `percent_difference` factor will calculate the difference between teh 30-day and the 60-day average closing price factors, and will then normalize this difference by the 60-day average closing price factor.

2. **Daily Returns Factor:** Create a factor that returns the daily percent change in closing price. To do this, use Ziplines built-in [DailyReturns](https://www.zipline.io/appendix.html#zipline.pipeline.factors.DailyReturns) factor.

Both factors should use the closing prices in the `USEquityPricing` dataset as inputs.

### Filters
In the code below, you will create a **Custom Filter**, `tradable_asset` that returns `True` for the top 20 securities of average dollar volume in a 30 day-window that have a latest closing price above $30. To do this, you will create two filters first and then combine them. For the first filter, you can use Zipline's built-in factor `AverageDollarVolume()`  and its method `.top()` to select the top 20 assets of average dollar volume in a 30 day-window. To get the latest closing price of the `USEquityPricing` dataset you can use `USEquityPricing.close.latest`. Finally you can combine these two filters by using the logical operator `&`.

In [4]:
# import resources
import pandas as pd
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.factors import DailyReturns


# Create the 30-day average closing price factor
mean_close_30 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 30)

# Create the 60-day average closing price factor
mean_close_60 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 60)

# Create the custom factor
percent_difference = (mean_close_30 - mean_close_60) / mean_close_60

# Create the daily returns factor
daily_ret = DailyReturns(inputs=[USEquityPricing.close])

# Create a filter for the top 20 securities of average dollar volume in a 30 day-window
top_20 = AverageDollarVolume(window_length = 30).top(20)

# Create a filter for the latest closing price above $30
above_30 = USEquityPricing.close.latest > 30

# Create a custum filter
tradable_asset = top_20 & above_30

### Add factors and filters to the pipeline
in the code below, add the factors and filters you created above to the pipeline

In [5]:
# Add the custom factor to the pipeline
pipeline.add(percent_difference, 'Percent Difference')

# Add the daily returns factor to the pipeline
pipeline.add(daily_ret, 'Daily Return')

# Add the custom filter to the pipeline
pipeline.add(tradable_asset, 'Tradable')

In [6]:
# Add the latest closing price to the pipeline
pipeline.add(USEquityPricing.close.latest, 'Latest Price')

### Run the Pipeline
In the code below run the pipeline for the dates given

In [7]:
# Set starting and end dates
start_date = pd.Timestamp('2014-01-06', tz='utc')
end_date = pd.Timestamp('2019-07-25', tz='utc')

# Run our pipeline for the given start and end dates
output = run_pipeline(pipeline, start_date, end_date)


# Display the pipeline output
output.head()

Daily Return  Latest Price  \
2014-01-06 00:00:00+00:00 Equity(2 [ARNC])       0.003894         10.57   
                          Equity(24 [AAPL])     -0.022034        541.04   
                          Equity(31 [ABAX])     -0.001747         40.01   
                          Equity(39 [DDC])      -0.011088         14.27   
                          Equity(41 [ARCB])      0.000000         33.83   

                                             Percent Difference Tradable  
2014-01-06 00:00:00+00:00 Equity(2 [ARNC])             0.041071    False  
                          Equity(24 [AAPL])            0.035897     True  
                          Equity(31 [ABAX])            0.009341    False  
                          Equity(39 [DDC])             0.014351    False  
                          Equity(41 [ARCB])            0.097416    False

In [8]:
output.tail()

Daily Return  Latest Price  \
2019-07-25 00:00:00+00:00 Equity(53360 [IHRT])     -0.036697         15.75   
                          Equity(53361 [IFS])      -0.024363         43.65   
                          Equity(53362 [MDLA])      0.109166         40.54   
                          Equity(53364 [WBIN])      0.001997         20.07   
                          Equity(53365 [AFYA])      0.115008         27.34   

                                                Percent Difference Tradable  
2019-07-25 00:00:00+00:00 Equity(53360 [IHRT])                 0.0    False  
                          Equity(53361 [IFS])                  0.0    False  
                          Equity(53362 [MDLA])                 0.0    False  
                          Equity(53364 [WBIN])                 0.0    False  
                          Equity(53365 [AFYA])                 0.0    False